In [1]:
from tqdm import tqdm, trange

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from subword_nmt.learn_bpe import learn_bpe
from subword_nmt.apply_bpe import BPE

from nltk.tokenize import WordPunctTokenizer

import torch
import torch.nn.functional as F
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'

from vocab import Vocab

In [2]:
train = pd.read_csv('train.csv')
train.head()

,title,description,subcategory,category,price,region,city,datetime_submitted,is_bad
0,Диван-кровать,Продаем диван-кровать. Удобный механизм - евро...,Мебель и интерьер,Для дома и дачи,7000.0,Россия,Москва,2019-06-01 00:00:15.180656,0
1,Кожух рулевой колонки Даф хф 91 4509834,Кожух рулевой колонки DAF XF 94 (60066004)/\n ...,Запчасти и аксессуары,Транспорт,2290.0,Россия,Москва,2019-06-01 00:00:44.317933,0
2,Дешёвый буст аккаунтов Dota 4,! Буст аккаунтов с ммр выше 1000ммр не беру ! ...,Предложение услуг,Услуги,200.0,Северная Осетия,Владикавказ,2019-06-01 00:00:50.249692,1
3,Телевизор sharp.Смарт тв.Интернет,Продам телевизор . Диагональ 450.наличие входа...,Аудио и видео,Бытовая электроника,25000.0,Калининградская область,Советск,2019-06-01 00:00:50.325799,1
4,Открытка-конверт,Открытки-конверты ручной работы/\nВыполнены в ...,Коллекционирование,Хобби и отдых,150.0,Ставропольский край,Ессентукская,2019-06-01 00:00:56.632655,0


In [3]:
val = pd.read_csv('val.csv')
val.head()

,title,description,subcategory,category,price,region,city,datetime_submitted,is_bad
0,Шины,Звонить 89425546881,Запчасти и аксессуары,Транспорт,2000.0,Тульская область,Огаревка,2019-10-10 00:00:25.200714,1
1,Продается мобильная перегородка с дверью,"Мобильная перегородка, предназначена для разгр...",Оборудование для бизнеса,Для бизнеса,10500.0,Вологодская область,Вологда,2019-10-10 00:03:11.527292,0
2,Комплект зимних шин на дисках 682/32/64,Шины зимние б/у Marshal Assimetric I”Zen KW 61...,Запчасти и аксессуары,Транспорт,4000.0,Россия,Москва,2019-10-10 00:05:07.193248,1
3,Кровать-трансормер Дакота сб-4085,"Продаю кровать-трансформер производства ""Столп...",Мебель и интерьер,Для дома и дачи,17000.0,Московская область,Химки,2019-10-10 00:05:58.165179,0
4,Honda VFR 800 2004 г.в,"Мот в отличном состоянии для своих лет, Родной...",Мотоциклы и мототехника,Транспорт,235000.0,Брянская область,Брянск,2019-10-10 00:06:19.231151,0


In [11]:
simple_tokenizer = WordPunctTokenizer()
def tokenize(x):
    return ' '.join(simple_tokenizer.tokenize(x.lower()))


train['description'].apply(tokenize).to_csv('./train_description', index=False, header=False)
val['description'].apply(tokenize).to_csv('./val_description', index=False, header=False)

In [12]:
learn_bpe(open('train_description'), open('bpe_rules', 'w'), num_symbols=8000)
bpe = BPE(open('./bpe_rules'))

with open('train.bpe', 'w') as f_out:
    for line in open('train_description'):
        f_out.write(bpe.process_line(line.strip()) + '\n')

with open('val.bpe', 'w') as f_out:
    for line in open('val_description'):
        f_out.write(bpe.process_line(line.strip()) + '\n')

In [106]:
train_text = np.array(open('./train.bpe').read().split('\n')[:-1])
val_text = np.array(open('./val.bpe').read().split('\n')[:-1])

In [6]:
inp_voc = Vocab.from_lines(train_text)

In [38]:
class myDataset(torch.utils.data.Dataset):
    def __init__(self, inp_voc, dataset_text, target):
        super().__init__
        self.inp_voc = inp_voc
        self.data = dataset_text
        self.target = target

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return inp_voc.to_matrix(self.data[idx]), torch.Tensor(self.target[idx])

In [39]:
train_dataset = myDataset(inp_voc, train_text, train['is_bad'])
val_dataset = myDataset(inp_voc, val_text, val['is_bad'])
trainLoader = torch.utils.data.DataLoader(train_dataset, 
                                          batch_size=32, 
                                          pin_memory=True, 
                                          shuffle=True, 
                                          num_workers=12)
valLoader = torch.utils.data.DataLoader(train_dataset, 
                                        batch_size=32, 
                                        pin_memory=True, 
                                        shuffle=False, 
                                        num_workers=12)

In [74]:
class BasicModel(nn.Module):
    def __init__(self, inp_voc, emb_size=128, hid_size=128):
        super().__init__()
        self.inp_voc = inp_voc
        self.hid_size = hid_size
        self.embedding = nn.Embedding(len(inp_voc), emb_size)
        self.gru = nn.GRU(emb_size, hid_size, batch_first=True)
        self.linear = nn.Linear(emb_size, 1)
        self.logits = nn.Sigmoid()
        
    def forward(self, inp):
        inp_emb = self.embedding(inp)
        batch_size = inp.shape[0]
        
        enc_seq, [last_state_but_not_really] = self.gru(inp_emb)

        lengths = (inp != self.inp_voc.eos_ix).to(torch.int64).sum(dim=1).clamp_max(inp.shape[1] - 1)
        last_state = enc_seq[torch.arange(len(enc_seq)), lengths]
        
        return self.logits(self.linear(last_state))

In [114]:
model = BasicModel(inp_voc).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-4)
loss = nn.BCELoss()
batch_size = 64

In [115]:
for _ in trange(25000):
    batch_ix = np.random.randint(len(train_text), size=batch_size)
    batch_inp = inp_voc.to_matrix(train_text[batch_ix]).to(device)
    batch_target = torch.Tensor(train['is_bad'].values[batch_ix]).to(device)


    probs = model(batch_inp).squeeze()
    opt.zero_grad()
    output = loss(probs, batch_target)
    output.backward()
    opt.step()

    
    if _ % 100 == 1:
        model.eval()
        with torch.no_grad():
            y_pred = np.array([])
            for i in range(0, len(val_text), batch_size):
                y_pred = np.append(y_pred, model(inp_voc.to_matrix(val_text[i: i + batch_size]).to(device)).squeeze().cpu().numpy())
        scores = []
        for category in val['category'].unique():
            scores.append(
                roc_auc_score(
                    val['is_bad'][val['category'] == category], 
                    y_pred[val['category'] == category]
                )
            )
        print(np.mean(scores))
    model.train()

  0%|          | 7/25000 [00:03<8:50:47,  1.27s/it] 

0.5482899172611481


  0%|          | 106/25000 [00:09<1:36:17,  4.31it/s]

0.6493605122559876


  1%|          | 208/25000 [00:15<1:26:11,  4.79it/s]

...

 99%|█████████▉| 24809/25000 [24:54<00:39,  4.88it/s]

0.9216926667089951


100%|█████████▉| 24907/25000 [25:00<00:15,  5.90it/s]

0.9232378458924062


100%|██████████| 25000/25000 [25:02<00:00, 16.64it/s]


In [116]:
torch.save(model.state_dict(), './model.torch')